## Making sure we can implement basic non-parameterised conv layer

In [1]:
# imports
import torch_geometric
import networkx as nx
import torch_geometric.nn
from data_gen import get_graph
import collections
import torch
import math

In [2]:
# get the first graph from the dataset
graph = get_graph()

# node features are (x,y,z) coordinates
node_features = graph.pos
# edge_index are dst and src pairs
edge_index = graph.edge_index

n_nodes = node_features.shape[0]
n_edges = edge_index.shape[1]
print(f"n_nodes: {n_nodes}, n_edges: {n_edges}")

n_nodes: 2518, n_edges: 15108


Get graph in a more convenient "adjacency list"-esque form:

In [3]:
# preprocess graph to get list of neighbors for each node
graph_dict = collections.defaultdict(list)
for i in range(edge_index.shape[1]):
    src = edge_index[0, i].item()
    dst = edge_index[1, i].item()
    graph_dict[dst].append(src)

In [4]:
# testing simple conv
simple_conv = torch_geometric.nn.SimpleConv(aggr="sum", combine_root="self_loop")
target_output = simple_conv(node_features, edge_index)
print(f"target output {target_output.shape} \n{target_output[0]}")

target output torch.Size([2518, 3]) 
tensor([-0.0642, -0.1196,  0.2208])


In [5]:
output = []
for i in range(node_features.shape[0]):
    new_features = torch.zeros_like(node_features[i])

    new_features += node_features[i]

    for neighbour in graph_dict[i]:
        new_features += node_features[neighbour]

    output.append(new_features) 

output = torch.stack(output)

print(f"output {output.shape} \n{output[0]}")



output torch.Size([2518, 3]) 
tensor([-0.0642, -0.1196,  0.2208])


In [6]:
result = torch.isclose(target_output, output).float().mean()
if result == 1:
    print("Success")
else:
    print(f"Failed - only {result} of the values are close to the target output.")


Success
